# 1. PSD(Power Spectral Density)

# 2. DE(Differential Entropy)

In [1]:
import os
import numpy as np
import math
import scipy.io as sio
from scipy.fftpack import fft,ifft

def DE_PSD(data,stft_para):
    '''
    compute DE and PSD
    --------
    input:  data [n*m]          n electrodes, m time points
            stft_para.stftn     frequency domain sampling rate
            stft_para.fStart    start frequency of each frequency band
            stft_para.fEnd      end frequency of each frequency band
            stft_para.window    window length of each sample point(seconds)
            stft_para.fs        original frequency
    output: psd,DE [n*l*k]        n electrodes, l windows, k frequency bands
    '''
    #initialize the parameters
    STFTN=stft_para['stftn']
    fStart=stft_para['fStart']
    fEnd=stft_para['fEnd']
    fs=stft_para['fs']
    window=stft_para['window']

    WindowPoints=fs*window

    fStartNum=np.zeros([len(fStart)],dtype=int)
    fEndNum=np.zeros([len(fEnd)],dtype=int)
    for i in range(0,len(stft_para['fStart'])):
        fStartNum[i]=int(fStart[i]/fs*STFTN)
        fEndNum[i]=int(fEnd[i]/fs*STFTN)

    #print(fStartNum[0],fEndNum[0])
    n=data.shape[0]
    m=data.shape[1]

    l = math.floor(m / WindowPoints)

    psd = np.zeros([n, l, len(fStart)])
    de = np.zeros([n, l, len(fStart)])
    #Hanning window
    Hwindow= np.hanning(WindowPoints)

    for j in range(n):
        for i in range(l):
            temp = data[j, WindowPoints*i : WindowPoints*(i+1)]
            Hdata = temp * Hwindow
            FFTdata = fft(Hdata, STFTN)
            magFFTdata = abs(FFTdata[0:int(STFTN/2)])
            for p in range(len(fStart)):
                E = 0
                for p0 in range(fStartNum[p], fEndNum[p]+1):
                    E = E + magFFTdata[p0] * magFFTdata[p0]
                
                E = E/(fEndNum[p]-fStartNum[p]+1)
                psd[j][i][p] = E
                de[j][i][p] = math.log(100*E,2)
    
    return psd,de


In [2]:
import simdkalman
from tqdm import tqdm

kf = simdkalman.KalmanFilter(
    state_transition = np.array([[1,1],[0,1]]),
    process_noise = np.diag([0.1, 0.01]),
    observation_model = np.array([[1,0]]),
    observation_noise = 1.0)

# names = ['chenyi', 'huangwenjing', 'huangxingbao', 'huatong', 'wuwenrui', 'yinhao']
names = ['chenyi']
for name in names:
    npy_path = r"D:\bcmi\EMBC\eeg_process\npydata_ff\{}_data.npy".format(name)

    subj_name = name
    print(subj_name)

    data_all = np.load(npy_path)

    # 6 * (1, 360, 62, 5, 5)
    data_psd = []
    data_de = []
    for b in range(6):
        print(b)
        data = data_all[b]
        # print(data.shape)

        stft_para = {'stftn':512, 'fStart':[1, 4, 8, 14, 31], 'fEnd':[4, 8, 14, 31, 50], 'fs':200, 'window':1}

        # 360 * (1, 62, 5, 5)
        data_psd_b = []
        data_de_b = []
        for case in tqdm(range(360)):
            psd, de = DE_PSD(data[case], stft_para)
            for c in range(62):
                psd[c] = psd[c].T
                de[c] = de[c].T
                for fb in range(5):
                    psd[c, fb] = kf.smooth(psd[c, fb]).observations.mean
                    de[c, fb] = kf.smooth(de[c, fb]).observations.mean
                
            data_psd_b.append(np.expand_dims(psd, axis=0))
            data_de_b.append(np.expand_dims(de, axis=0))

        data_psd_b = np.concatenate(data_psd_b, axis=0)
        data_de_b = np.concatenate(data_de_b, axis=0)

        data_psd.append(np.expand_dims(data_psd_b, axis=0))
        data_de.append(np.expand_dims(data_de_b, axis=0))

    data_psd = np.concatenate(data_psd, axis=0)
    data_de = np.concatenate(data_de, axis=0)

    print(data_psd.shape)
    print(data_de.shape)

    out_dir = './npydata_ff'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    np.save(out_dir+'/{}_data_psd.npy'.format(subj_name), data_psd)
    np.save(out_dir+'/{}_data_de.npy'.format(subj_name), data_de)

chenyi


  0%|          | 0/360 [00:00<?, ?it/s]

0


  0%|          | 0/360 [00:00<?, ?it/s]

1


...

4


  0%|          | 0/360 [00:00<?, ?it/s]

5


100%|██████████| 360/360 [03:34<00:00,  1.68it/s]


(6, 360, 62, 5, 5)
(6, 360, 62, 5, 5)


In [ ]:
import simdkalman
import matplotlib.pyplot as plt

kf = simdkalman.KalmanFilter(
    state_transition = np.array([[1,1],[0,1]]),
    process_noise = np.diag([0.1, 0.01]),
    observation_model = np.array([[1,0]]),
    observation_noise = 1.0)

npy_path = r"D:\bcmi\EMBC\eeg_process\npydata_new\chenyi_data.npy"

data = np.load(npy_path)[0][0]

stft_para = {'stftn':512, 'fStart':[1, 4, 8, 14, 31], 'fEnd':[4, 8, 14, 31, 50], 'fs':200, 'window':1}

psd, de = DE_PSD(data, stft_para)
print(psd[0][0])
for c in range(62):
    psd[c] = psd[c].T
    de[c] = de[c].T
    for fb in range(5):
        psd[c, fb] = kf.smooth(psd[c, fb]).observations.mean
        de[c, fb] = kf.smooth(de[c, fb]).observations.mean
print(psd[0][0])

# data1 = 
# smoothed1 = kf.smooth(data1)
# print(smoothed1)

# x = np.arange(5)
# plt.plot(x, data1, label='data1')
# plt.plot(x, smoothed1.observations.mean, label='smoothed1')
# plt.legend()

In [ ]:
a = np.ones((2, 3, 3))
a

In [ ]:
a[0, 0] = [2, 2, 2]
a[1, 0] = [3, 3, 3]
a

In [ ]:
a[0].T

In [ ]:
a[0] = a[0].T
a